## Siamese NN

In [1]:
import os
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
from SNN.siamese_network import *
from SNN.siamese_network_parse import PrepareDataSNN
from sklearn.model_selection import train_test_split


2025-01-08 23:51:48.374474: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-08 23:51:49.059470: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Preparar la data

In [22]:
data = PrepareDataSNN()

# Ruta para guardar los datos
data_dir = "./data_snn"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de entrenamiento y validación desde archivo...")
    data = np.load(data_file)
    X1_train, X1_val = data["X1_train"], data["X1_val"]
    X2_train, X2_val = data["X2_train"], data["X2_val"]
    y_train, y_val = data["y_train"], data["y_val"]
else:
    # Crear los conjuntos de datos y guardarlos
    print("Dividiendo los datos y guardándolos...")
    data_a, data_b, labels = PrepareDataSNN().process()
    X1_train, X1_val, X2_train, X2_val, y_train, y_val = train_test_split(
        data_a, data_b, labels, test_size=0.2, random_state=42
    )
    np.savez(data_file, X1_train=X1_train, X1_val=X1_val, 
                X2_train=X2_train, X2_val=X2_val, 
                y_train=y_train, y_val=y_val)

# Verificar las formas de los arrays
print("Formas de los conjuntos de entrenamiento:")
print(f"X1_train: {X1_train.shape}")
print(f"X2_train: {X2_train.shape}")
print(f"y_train: {y_train.shape}")

print("\nFormas de los conjuntos de validación:")
print(f"X1_val: {X1_val.shape}")
print(f"X2_val: {X2_val.shape}")
print(f"y_val: {y_val.shape}")


Dividiendo los datos y guardándolos...
Formas de los conjuntos de entrenamiento:
X1_train: (720, 65)
X2_train: (720, 65)
y_train: (720,)

Formas de los conjuntos de validación:
X1_val: (180, 65)
X2_val: (180, 65)
y_val: (180,)


## Inicializar la red siamesa, entrenar y guardar el modelo

In [20]:
import keras
# Inicializar y entrenar la red
input_shape = (65,)  # Asumiendo que tus vectores de características tienen 65 elementos
siamese_net = SiameseNeuralNetwork(input_shape)
history = siamese_net.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val))

# Guardar el modelo
siamese_net._save_model("model_1")
# keras.saving.save_model(siamese_net, './models/a.keras')

Epoch 1/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5202 - loss: 0.7496 - val_accuracy: 0.6444 - val_loss: 0.6646
Epoch 2/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6287 - loss: 0.6473 - val_accuracy: 0.6833 - val_loss: 0.6239
Epoch 3/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6824 - loss: 0.6196 - val_accuracy: 0.7056 - val_loss: 0.5800
Epoch 4/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7403 - loss: 0.5642 - val_accuracy: 0.7333 - val_loss: 0.5410
Epoch 5/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8051 - loss: 0.4812 - val_accuracy: 0.7500 - val_loss: 0.5146
Epoch 6/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8245 - loss: 0.4092 - val_accuracy: 0.7611 - val_loss: 0.5127
Epoch 7/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8493 - loss: 0.3952 - val_accuracy: 0.7500 - val_loss: 0.5166
Epoch 8/20
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8313 - loss: 0.3840 - val_accuracy: 0.7778 - val_loss:

## Cargar el modelo guardado

In [24]:
model = load_model('./models/best_model_all_dataset_val.keras', custom_objects={'L1Distance': L1Distance})

model.summary()

Model: "functional_1080"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_457     │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_458     │ (None, 65)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential_152      │ (None, 128)       │    198,016 │ input_layer_457[… │
│ (Sequential)        │                   │            │ input_layer_458[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ l1_distance_152     │ (None, 128)       │          0 │ sequential_152[0… │
│ (L1Distance)        │                   │            │ sequential_152[1… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_611 (Dense)   │ (None, 1)         │        129 │ l1_distance_152[… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 594,437 (2.27 MB)

 Trainable params: 198,145 (774.00 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 396,292 (1.51 MB)

## Preparar la nueva data de validacion (Proyectos de C#)

In [29]:

# Ruta para guardar los datos
data_dir = "./data_val"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

data_file = os.path.join(data_dir, "dataset_split.npz")

if os.path.exists(data_file):
    # Cargar los datos si ya están guardados
    print("Cargando conjuntos de validación desde archivo...")
    data = np.load(data_file)
    X1_new, X2_new, y_new = data["X1_new"], data["X2_new"], data["y_new"]
else:
    print("Procesando los datos de validación y guardándolos...")
    data_val = PrepareDataSNN("data/features_vect_val/")
    X1_new, X2_new, y_new = data_val.process()
    
    # Guardar los datos procesados
    np.savez(data_file, X1_new=X1_new, X2_new=X2_new, y_new=y_new)


# Verificar las formas de los arrays
print("Formas de los conjuntos de validación:")
print(f"X1_new: {X1_new.shape}")
print(f"X2_new: {X2_new.shape}")
print(f"y_new: {y_new.shape}")


Cargando conjuntos de validación desde archivo...
Formas de los conjuntos de validación:
X1_new: (116, 65)
X2_new: (116, 65)
y_new: (116,)


## Evaluar el modelo con los datos nuevos

In [30]:
m = model.evaluate([X1_new, X2_new], y_new)

predictions = (model.predict([X1_new, X2_new]) > 0.5).astype(int).flatten()
        
# Calcular la matriz de confusión
tn, fp, fn, tp = confusion_matrix(y_new, predictions).ravel()

# Calcular métricas adicionales
precision = precision_score(y_new, predictions)
recall = recall_score(y_new, predictions)
f1 = f1_score(y_new, predictions)

# Imprimir las métricas adicionales
print(f"Verdaderos Positivos (TP): {tp}")
print(f"Falsos Positivos (FP): {fp}")
print(f"Verdaderos Negativos (TN): {tn}")
print(f"Falsos Negativos (FN): {fn}")
print(f"Precisión (Precision): {precision:.4f}")
print(f"Exhaustividad (Recall): {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7486 - loss: 0.3838 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 947us/step
Verdaderos Positivos (TP): 6
Falsos Positivos (FP): 4
Verdaderos Negativos (TN): 83
Falsos Negativos (FN): 23
Precisión (Precision): 0.6000
Exhaustividad (Recall): 0.2069
F1-Score: 0.3077


In [31]:
metrics = siamese_net.evaluate(X1_new, X2_new, y_new)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7517 - loss: 0.3895 
Pérdida en los nuevos datos: 0.3749
Accuracy en los nuevos datos: 0.7672
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
Verdaderos Positivos (TP): 8
Falsos Positivos (FP): 6
Verdaderos Negativos (TN): 81
Falsos Negativos (FN): 21
Precisión (Precision): 0.5714
Exhaustividad (Recall): 0.2759
F1-Score: 0.3721


## Hacer una predicción

In [11]:
code1_features = np.random.rand(65)  # Ejemplo de vector de características
code2_features = np.random.rand(65)  # Ejemplo de vector de características

similarity = siamese_net.predict_similarity(code1_features, code2_features)

# Interpretar el resultado
if similarity < 0.5:
    print(f"Los códigos son similares con una probabilidad de {1-similarity:.2f}")
else:
    print(f"Los códigos son diferentes con una probabilidad de {similarity:.2f}")

print(f"La similitud entre los dos códigos es: {similarity}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
Los códigos son similares con una probabilidad de 0.64
La similitud entre los dos códigos es: 0.35796594619750977


## Buscando la mejor semilla para el modelo

In [23]:
import os
import tensorflow as tf
import numpy as np
import random

# Definir un rango de semillas
SEEDS = range(50)  # Probar con semillas del 0 al 49

# Variables para almacenar los mejores resultados
best_accuracy = 0
best_precision = 0
best_seed = None
best_model = None

# Datos de entrenamiento y validación
# Asegúrate de tener `X1_train`, `X2_train`, `y_train`, `X1_val`, `X2_val`, `y_val` ya definidos.

for seed in SEEDS:
    print(f"Probando con semilla: {seed}")
    
    # Configurar la semilla
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    
    # Crear y entrenar el modelo
    snn = SiameseNeuralNetwork(input_shape=(65,), learning_rate=0.001)
    snn.train(X1_train, X2_train, y_train, validation_data=(X1_val, X2_val, y_val), epochs=10, batch_size=32)
    
    # Evaluar el modelo
    metrics = snn.evaluate(X1_new, X2_new, y_new)
    accuracy = metrics["accuracy"]
    precision = metrics["precision"]
    
    # Comparar con los mejores resultados
    if accuracy > best_accuracy or (accuracy == best_accuracy and precision > best_precision):
        best_accuracy = accuracy
        best_precision = precision
        best_seed = seed
        best_model = snn.model  # Guardar el modelo actual como el mejor

# Imprimir los resultados
print(f"Mejor semilla: {best_seed}")
print(f"Mejor accuracy: {best_accuracy:.4f}")
print(f"Mejor precisión: {best_precision:.4f}")

# Guardar el mejor modelo
tf.keras.config.enable_unsafe_deserialization()
best_model.save("models/best_model_all_dataset_val.keras")


Probando con semilla: 0
Epoch 1/10


/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5432 - loss: 0.7036 - val_accuracy: 0.6500 - val_loss: 0.6681
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5322 - loss: 0.6958 - val_accuracy: 0.7444 - val_loss: 0.6397
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6624 - loss: 0.6428 - val_accuracy: 0.8000 - val_loss: 0.5844
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7253 - loss: 0.5735 - val_accuracy: 0.8056 - val_loss: 0.5328
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7338 - loss: 0.5214 - val_accuracy: 0.8056 - val_loss: 0.4894
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8178 - loss: 0.4494 - val_accuracy: 0.7889 - val_loss: 0.4928
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8358 - loss: 0.3899 - val_accuracy: 0.7722 - val_loss: 0.5170
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8339 - loss: 0.3854 - val_accuracy: 0.7944 - val_loss: 0.4914
Epo

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5196 - loss: 0.7216 - val_accuracy: 0.6278 - val_loss: 0.6613
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6173 - loss: 0.6643 - val_accuracy: 0.7389 - val_loss: 0.5953
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6796 - loss: 0.6172 - val_accuracy: 0.7944 - val_loss: 0.5469
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7642 - loss: 0.5374 - val_accuracy: 0.8000 - val_loss: 0.4978
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7866 - loss: 0.4882 - val_accuracy: 0.7889 - val_loss: 0.4905
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7838 - loss: 0.4545 - val_accuracy: 0.8056 - val_loss: 0.5095
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8367 - loss: 0.4004 - val_accuracy: 0.8056 - val_loss: 0.5063
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8259 - loss: 0.3571 - val_accuracy: 0.7944 - val_loss: 0.4988
Epo

...

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5361 - loss: 0.7076 - val_accuracy: 0.7222 - val_loss: 0.6280
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5903 - loss: 0.6537 - val_accuracy: 0.8111 - val_loss: 0.5684
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7330 - loss: 0.5683 - val_accuracy: 0.8111 - val_loss: 0.5078
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7935 - loss: 0.4833 - val_accuracy: 0.7889 - val_loss: 0.4839
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8288 - loss: 0.4132 - val_accuracy: 0.7556 - val_loss: 0.4851
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8413 - loss: 0.3940 - val_accuracy: 0.8000 - val_loss: 0.4602
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8447 - loss: 0.3661 - val_accuracy: 0.7944 - val_loss: 0.4740
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8562 - loss: 0.3519 - val_accuracy: 0.8111 - val_loss: 0.4838
Epo

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5074 - loss: 0.7193 - val_accuracy: 0.6611 - val_loss: 0.6727
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6091 - loss: 0.6670 - val_accuracy: 0.7833 - val_loss: 0.6297
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5755 - loss: 0.6727 - val_accuracy: 0.8222 - val_loss: 0.5932
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6964 - loss: 0.6047 - val_accuracy: 0.8222 - val_loss: 0.5427
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7636 - loss: 0.5334 - val_accuracy: 0.8222 - val_loss: 0.5002
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8383 - loss: 0.4345 - val_accuracy: 0.8167 - val_loss: 0.5088
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8254 - loss: 0.4010 - val_accuracy: 0.8000 - val_loss: 0.5195
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8437 - loss: 0.3757 - val_accuracy: 0.7778 - val_loss: 0.5576
Epo

/home/chony/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5055 - loss: 0.7307 - val_accuracy: 0.7667 - val_loss: 0.6450
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5796 - loss: 0.6758 - val_accuracy: 0.8111 - val_loss: 0.5900
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6522 - loss: 0.6186 - val_accuracy: 0.8056 - val_loss: 0.5388
Epoch 4/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7490 - loss: 0.5262 - val_accuracy: 0.8000 - val_loss: 0.5099
Epoch 5/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8098 - loss: 0.4463 - val_accuracy: 0.8000 - val_loss: 0.4991
Epoch 6/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8320 - loss: 0.3913 - val_accuracy: 0.7556 - val_loss: 0.5140
Epoch 7/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8204 - loss: 0.4116 - val_accuracy: 0.7722 - val_loss: 0.5209
Epoch 8/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8350 - loss: 0.3906 - val_accuracy: 0.7889 - val_loss: 0.5144
Epo